In [ ]:
import sys
from pathlib import Path
from typing import BinaryIO

import numpy as np
import numpy.typing as npt

In [ ]:
# There are not really lines in binary files
# Means, read one entry...
def readline(
    f: BinaryIO, with_distances: bool, is_8ply: bool
) -> tuple[int | None, int | None]:
    """Reads a single entry from a binary file representing game positions.

    Args:
        f (BinaryIO): The file object to read from.
        with_distances (bool): Indicates if the distance (score) is stored separately.
        is_8ply (bool): Indicates if positions are stored in 8-ply format (3 bytes).

    Returns:
        tuple[int | None, int | None]: A tuple containing the Huffman position and
                                       the score, or (None, None) if EOF.
    """
    bytes_position = 3 if is_8ply else 4
    x = f.read(bytes_position)
    if not x:
        return None, None  # EOF
    huffman_position = int.from_bytes(x, byteorder="big", signed=not is_8ply)
    if with_distances:  # last byte contains the score (read one more byte)
        score = int.from_bytes(f.read(1), byteorder="big", signed=True)
    else:  # last 2 bits indicate the score
        score = (huffman_position & 3) * (-1)  # last 2 bits are the score
        huffman_position = (huffman_position // 4) * 4  # mask out last 2 bits
    return huffman_position, score

In [ ]:
def read_book(
    file: str, with_distances: bool = True, is_8ply: bool = False
) -> list[tuple[int, int]]:
    """Reads an entire book file containing positions and scores into memory.

    Args:
        file (str): Path to the binary book file.
        with_distances (bool, optional): Indicates if distances are stored.
                                         Defaults to True.
        is_8ply (bool, optional): Indicates if positions are in 8-ply format.
                                  Defaults to False.

    Returns:
        list[tuple[int, int]]: A list of tuples with positions and corresponding scores.
    """
    # one could also use a dictionary, for the sake of simplicity
    # However, the memory requirements for a dictionary are 5x higher
    book = []
    with Path.open(file, "rb") as f:
        while True:
            position, score = readline(f, with_distances, is_8ply)
            if position is None:
                break  # End of file (EOF) reached
            book.append((position, score))
    return book

In [ ]:
# One could also use some libraries instead...
def binary_search(book: list[tuple[int, int]], huffman_position: int) -> int | None:
    """Performs binary search to find the score for a given Huffman position.

    Args:
        book (list[tuple[int, int]]): Sorted list of positions and scores.
        huffman_position (int): The Huffman-encoded position to search for.

    Returns:
        int | None: The score associated with the position, or None if not found.
    """
    left = 0
    right = len(book) - 1
    while right >= left:
        mid = (left + right + 1) // 2
        p = book[mid][0]
        if p == huffman_position:
            return book[mid][1]  # Found! return the value for this position
        if p > huffman_position:
            right = mid - 1
        else:  # p < huffman_position
            left = mid + 1
    return None  # should not happen, ideally

In [ ]:
# Not extremly efficient... Might have to re-write this to get better performance
def to_huffman(board: npt.NDArray) -> int:
    """Converts a game board into its Huffman-encoded integer representation.

    Args:
        board (npt.NDArray): The Connect-4 board as a 2D numpy array.

    Returns:
        int: The Huffman-encoded integer representing the game board.
    """
    b = "0b"
    # Run through the columns, from left to right, and from bottom to top
    for col in range(board.shape[1]):  # 0-6
        for row in range(board.shape[0]):  # 0-5
            if board[row, col] == 0:  # no more pieces in this column
                b += "0"  # separator bit (indicates that there are no more pieces
                # in this column)
                break
            if board[row, col] == 1:  # Piece of player1 (starting player)
                b += "10"  # 2 bit for encoding piece of player 1
            else:  # Player 2
                b += "11"  # also 2 bit for encoding piece of player 2
            if row == 5:
                b += "0"  # add separator symbol, if column is completely full
    b = b + "0"  # add one bit to fill up to a full byte (24 bit/32 bit)
    b_int = int(b, 2)
    if (b[2] == "1") and len(
        b
    ) > 32:  # since we have signed integers in the book, use seconds complement
        b_int -= 2 << 31  # 2**32
    return b_int

In [ ]:
def get_book_value(
    board: npt.NDArray, book: list[tuple[int, int]], with_distances: bool = True
) -> int | None:
    """Retrieves the score for a given board position from the book.

    Args:
        board (npt.NDArray): The Connect-4 board as a numpy array.
        book (list[tuple[int, int]]): The book containing Huffman positions and scores.
        with_distances (bool, optional): Indicates if distances are stored.
                                         Defaults to True.

    Returns:
        int | None: The score for the board position or None if not found
                    (1 for unlisted positions without distances).
    """
    # first try this position
    p = to_huffman(board)
    val = binary_search(book, p)
    if val is not None:
        return val
    # Did not find position. Look for the mirrored equivalent
    p = to_huffman(np.flip(board, axis=1))
    val = binary_search(book, p)
    if (
        not with_distances and val is None
    ):  # only for the 8-ply and 12-ply database without distances
        val = 1  # if a position is not in the database, then this means that
        # player 1 wins
    return val

In [ ]:
# takes some time in Python
book = read_book(
    "../src/bitbully/assets/book_depth12_distances.dat",
    with_distances=True,
    is_8ply=False,
)  # 12-ply database with winning distances
book2 = read_book(
    "../src/bitbully/assets/book_depth12.dat", with_distances=False, is_8ply=False
)  # 12-ply database with simple win-loss-draw values
book3 = read_book(
    "../src/bitbully/assets/book_depth8.dat", with_distances=False, is_8ply=True
)  # 8-ply database with simple win-loss-draw values

In [ ]:
print("Number of entries in book:", len(book))  # should be 4200899
print(
    "Estimated size of book in RAM (in MB):", round(sys.getsizeof(book) / 1024.0**2, 2)
)
print("Number of entries in book2:", len(book2))  # should be 1735945
print(
    "Estimated size of book2 in RAM (in MB):",
    round(sys.getsizeof(book2) / 1024.0**2, 2),
)  # This one should be smaller, since it does not contain winning positions of player 1
print("Number of entries in book3:", len(book3))  # should be
print(
    "Estimated size of book3 in RAM (in MB):",
    round(sys.getsizeof(book3) / 1024.0**2, 2),
)  # This one should be smaller, since it does not contain winning positions of player 1

In [ ]:
# Do a Test: each position in book, should also be in book2
# And the values should match
for b in book:
    val2 = binary_search(book2, b[0])  # look into book2 and find position of b
    if val2 is None:  # If there is no entry in book2, it means that player 1 will win
        assert b[1] > 0
    elif val2 == 0:
        assert b[1] == 0  # draw is zero in both cases
    elif val2 < 0:
        assert b[1] < 0  # in both cases the value should be negative

In [ ]:
# Make a example position.
# Player to move first: P1
# Player to move second: P2

# Empty board with 6 rows (1st index) and 7 columns (2nd index)
# A[0,0] is the cell in the bottom left
A = np.zeros((6, 7), dtype=np.int32)
A[0, 3] = 1  # Move 1: P1 in Center Column
A[1, 3] = 2  # Move 2: P2 in Center Column
A[2, 3] = 1  # Move 3: P1 in Center Column
A[3, 3] = 2  # Move 4: P2 in Center Column
A[4, 3] = 1  # Move 5: P1 in Center Column
A[0, 1] = 2  # Move 6: P2 in 2nd Column
A[1, 1] = 1  # Move 7: P1 in 2nd Column
A[2, 1] = 2  # Move 8: P2 in 2nd Column
B = A.copy()  # Keep this position with 8 pieces for later
A[3, 1] = 1  # Move 9: P1 in 2nd Column
A[0, 5] = 2  # Move 10: P2 in 5th Column
A[1, 5] = 1  # Move 11: P1 in 5th Column
A[2, 5] = 2  # Move 12: P2 in 5th Column
# Now we can access the database...

print(np.flip(A, axis=0))  # flip, for correct "visualization"

In [ ]:
print(
    get_book_value(A, book, with_distances=True)
)  # Player 1 will win after 100-71 = 29 moves (larger score is better)
# If Player 2 will win, the score will be negative: -100+(distance to win).
# smaller score is better.
# If there is no winner (draw), the score will be 0
print(
    get_book_value(A, book2, with_distances=False)
)  # should be 1, since player 1 wins after 29 moves

# !! DO NOT DO THIS !! THE book3 only contains positions with 8 stones.
# If a position is not found, it will
# be assumed that the position is a win for player 1 (since the data
# base does not contain winning positions for player 1)
print(get_book_value(A, book3, with_distances=False))  # DO NOT DO THIS!

In [ ]:
# This is allowed, since this position contains exactly 8 stones
B = A.copy()  # Keep this position with 8 pieces for later
print(
    get_book_value(B, book3, with_distances=False)
)  # should be 1, since player 1 wins after 29 moves

In [ ]:
# Build another position with 8 stones, where player 1 will loose
C = np.zeros((6, 7), dtype=np.int32)
C[0, 0] = 1  # Move 1: P1 in leftmost Column
C[0, 3] = 2  # Move 2: P2 in Center Column
C[1, 3] = 1  # Move 3: P1 in Center Column
C[2, 3] = 2  # Move 4: P2 in Center Column
C[3, 3] = 1  # Move 5: P1 in Center Column
C[4, 3] = 2  # Move 6: P2 in Center Column
C[5, 3] = 1  # Move 7: P1 in Center Column
C[0, 5] = 2  # Move 8: P2 in 6th Column
print(np.flip(C, axis=0))  # flip, for correct "visualization"

In [ ]:
print(get_book_value(C, book3, with_distances=False))  # should be -1

In [ ]:
# One last position for testing purposes
D = np.zeros((6, 7), dtype=np.int32)
D[0, 0] = D[0, 5] = D[1, 5] = D[2, 5] = D[4, 5] = D[2, 6] = 1
D[3, 5] = D[0, 6] = D[1, 6] = D[3, 6] = D[4, 6] = D[5, 6] = 2
print(np.flip(D, axis=0))  # flip, for correct "visualization"

In [ ]:
print(
    get_book_value(D, book, with_distances=True)
)  # Player 1 will win after 25 moves (100-75)

In [ ]:
book[0]